<a href="https://colab.research.google.com/github/JuliaKovalchook/Ukraine-TopoJSON/blob/main/Instructions_for_creating_%D1%81ontour_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Openstreetmap

Openstreetmap містить інформацію щодо координат і багато іншої географічної інформації щодо всіх країн, громад, сіл і навіть озер чи ставків.
Існує як і API для витягу даних так і платформа, яка одразу допомагає візуалізувати запит для витягу дані.  
<br>
Якщо ви бажаєте витягнути лише декількох файлів з інформацією або ви ще не надто впевнено себе відчуваєте з API, Overpass turbo є найкращим рішенням оскільки візуалізація одразу показує які дані є зайві, а яких не хватає.


Ми розглянемо використання двох інструментів:

1.   [Overpass turbo](https://overpass-turbo.eu/)
2.   Overpass API    ([документація](https://wiki.openstreetmap.org/wiki/Overpass_API))






### Overpass turbo

Приклад коду який витягуватиме дані всіх громад Київська області


```
[out:json][timeout:25];  
area['admin_level'='4']['name:uk'='Київська область'];
(rel['admin_level'='7'](area););
out geom;
```


Перший рядок `[out:json]` вказує дані якого формату ми бажаємо витягнути <br>

Також можливі варінати витяг даних формату xml `[out:xml]` чи csv, при цьому для файлів формату csv обовязково вказувати теги, які ми бажаємо витягнути `[out:csv(::type,::id,name,"name:en")]` та [інші](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL#Output_format_(out:)


`[timeout:25]` обмежує час виконання запиту до 25 секунд. Якщо щось піде не так, то ліміт гарантує, що сервер не витрачатиме час.


---

Другий рядок `area['admin_level'='4']['name:uk'='Київська область']` фільтрує дані вказуючи, що ми хочемо отримати  дані, які знаходяться в межах  де admin_level=4 (тобто область) і називається "Київська область". Якщо ми бажаємо отримати дані по всій країні, то варто змінити рядок на `area['admin_level'='2']['name:uk'='Україна']`

---

Третій рядок `(rel['admin_level'='7'](area);)` вказує що ми хочемо отримати, в даному випадку це `rel` (відносини), інші можливі ваіранти це `node` і `way`. Географічна інформація про широту/довжину завжди зберігається у вузлах. Області з понад 2000 вузлів не можуть бути представлені шляхом, ця функція вимагатиме складнішої багатополігональної структури відсини, тому в даному випадку використовується саме сутність rel (відносини).  <br> `'admin_level'='7'` вкзаує, що саме ми хочемо витягнути села, міста, райони, в даному випадку це громади.

---

Нарешті, `out geom` виводить відношення (включаючи їх дочірні шляхи та вузли). Існують багато інших варіантів оператору виходу, вони гарно описані [тут](https://dev.overpass-api.de/overpass-doc/en/targets/formats.html)


Приклад коду який витягуватиме дані всіх областей України

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='4'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх районів України* (після реформи децентралізації)

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='6'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх громад України*

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='7'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх районів Києва

```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='10'](area););
out geom;
```
**(за виключенням тимчасово окупованих тереторій)*


---




Місто Київ не є громадою, проте якщо ми витягуватимео дані по всіх громадах Київської області, то на мапі України в нас буде пусте місце яке належитеме місту Києву. Отже припустимо ми бажаємо отримати всі громади Київської області і міста Київ.
Спочатку спробуємо отримати дані  Києва. Нижче наведений запит, який також поверне Київську область, оскільки admin_level для міста Києва і області одинаковий

```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4'](area););
out geom;
```

Тож варто вказати, що ми шукаємо саме місто  `['place'='city']`

```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4']['place'='city'](area););
out geom;
```

Після цього, якщо запити не суперечать одне одному ми можемо їх просто обєднати
```
[out:json];
area['admin_level'='4']['name:uk'='Київська область'];
(rel['admin_level'='7'](area););
out geom;
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4']['place'='city'](area););
out geom;
```








Для конвертації файлу з GeoJSON до TurboJson  можна використати будь який онлайн сай, наприклад:
https://mapshaper.org/